# Library

In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

# File Load
- 4등급_차량_리스트_양식_result
- 정기
- 정밀

In [29]:
ods_fold = 'D:/data/big2/ODS'
fold = 'D:/data/big2/BD1_4등급_데이터_점검1'

In [66]:
# about 11s
file_name = '4등급_차량_리스트_양식_result.csv'
df = pd.read_csv(os.path.join(fold, file_name), encoding='cp949', low_memory=False)

In [4]:
# about 3m 33s
file_name = '[ODS]정기검사(2022.12.29).csv'
ins = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)

In [5]:
# about 5m 55s
file_name = '[ODS]정밀검사(2022.12.29).csv'
dins = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)

In [6]:
df.shape, ins.shape, dins.shape

((1511251, 49), (22006426, 39), (28682228, 41))

In [31]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무'],
      dtype='object')

In [8]:
ins.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간'],
      dtype='object')

In [9]:
dins.columns

Index(['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '차량용도', '최대출력허용치', '최대출력측정값', '검사접수번호', '검사소명', '재검사기간',
       '검사 검사소코드', '검사유효기간'],
      dtype='object')

# Preprocessing

## 전처리 : 4등급_차량_리스트_양식_result
- 컬럼명 정리
    - '차종' : '차종_원본', '인증번호':'배출가스인증번호'
- ~~중복 차대번호 처리 : '최초등록일', '검사유효일, 내림차순 정렬(ascending=[False, False])~~
- 배출가스등급 : 4등급
- 연료 : 경유차

In [32]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무'],
      dtype='object')

In [67]:
df = df.rename(columns={'차종':'차종_원본', '인증번호':'배출가스인증번호'})
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종_원본', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무'],
      dtype='object')

In [68]:
df.shape, len(df['차대번호'].unique())

((1502235, 28), 1502213)

In [69]:
# df = df.sort_values(['최초등록일', '검사유효일'], ascending=[False, False]).drop_duplicates('차대번호').reset_index(drop=True)
# df.shape

In [70]:
df['등급'].value_counts(dropna=False)

4    1502235
Name: 등급, dtype: int64

In [71]:
df['연료'].value_counts(dropna=False)

경유             1153813
휘발유             322983
LPG(액화석유가스)      25400
기타연료                37
알코올                  1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [72]:
df.shape

(1502235, 28)

In [73]:
df = df[df['연료'] == '경유'].reset_index(drop=True)
df.shape

(1153813, 28)

## 전처리 : 정기, 정밀 검사
- 정기, 정밀 검사 필요한 컬럼만 추출
- 정기, 정밀 검사 샘플에 '정기, 정밀' 검사 표기
- 정기, 정밀 통합하여 중복 차대번호 제거(최신 '검사일자'만 사용)

In [16]:
len(ins.columns), len(dins.columns)

(39, 41)

In [17]:
col = ['사용연료', '주행거리', '차명', '차량번호', '차대번호', '차종', '차량연식', '엔진형식', '검사방법',
       '검사일자', '검사종류', '검사판정시각', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6',]
len(col)

33

In [18]:
# about 30s
ins = ins[col]
dins = dins[col]
ins.shape, dins.shape

((22006426, 33), (28682228, 33))

In [19]:
ins['정기정밀'] = '정기'
dins['정기정밀'] = '정밀'

In [20]:
# about 13s
ins.shape, len(ins['차대번호'].unique())

((22006426, 34), 12601262)

In [21]:
# 20s
dins.shape, len(dins['차대번호'].unique())

((28682228, 34), 15367596)

In [22]:
# about 
tin = pd.concat([ins, dins], ignore_index=True)
tin.shape

(50688654, 34)

In [23]:
tin.shape[0] == ins.shape[0] + dins.shape[0]

True

In [24]:
tin = tin.sort_values('검사일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
tin.shape

(22044208, 34)

## 전처리 : 등록&제원과 정기,정밀 병합
- 4등급 경유 등록&제원 : df
- 최신 검사일자 중복 차대번호 제거 정기, 정밀 : tin
- 병합 : total

In [74]:
# about 45s
total = df.merge(tin, on='차대번호', how='left')
total.shape

(1153813, 61)

# Analysis
- '배출가스인증번호'별 '무부하매연판정1'별 '차대번호' 개수
- '배출가스인증번호'별 '검사방법', '무부하매연판정1'별 '차대번호' 개수
- '배출가스인증번호'별 '검사방법', '무부하매연판정1'별 '무부하매연측정치1'의 통계

In [75]:
total['검사종류'].value_counts(dropna=False)

종합검사          731612
정기검사          171096
2부재검사(종합)      87447
종합검사(경과)       60618
재검사(2부재검사)     31210
NaN            22637
1부재검사(종합)      18846
재검사(1부재검사)     18420
정기(경과)검사       11927
Name: 검사종류, dtype: int64

In [76]:
total['검사방법'].value_counts(dropna=False)

부하검사(KD-147)        798238
무부하검사(급가속)          289263
부하검사(LUG DOWN)       43669
NaN                  22637
부하검사(ASM-Idling)         3
무부하검사(정지가동)              2
무부하검사(TSI)               1
Name: 검사방법, dtype: int64

In [77]:
total['무부하매연판정1'].value_counts(dropna=False)

Y      1087888
N        34609
NaN      31316
Name: 무부하매연판정1, dtype: int64

In [78]:
total['무부하매연측정치1'].value_counts(dropna=False)

0.00      276379
1.00       84223
2.00       70398
3.00       63318
4.00       62680
           ...  
0.01           1
99.90          1
2.50           1
135.00         1
103.00         1
Name: 무부하매연측정치1, Length: 111, dtype: int64

In [79]:
total['무부하매연측정치1'].isnull().sum()

31312

In [80]:
total.groupby(['무부하매연판정1'], dropna=False)['차대번호'].agg(['count'])

,count
무부하매연판정1,
N,34609
Y,1087888
NaN,31316


In [96]:
gas_total_group = total.groupby(['무부하매연판정1', '검사방법'], dropna=False)['차대번호'].agg(['count']).unstack()
gas_total_group

count                                                       \
검사방법     무부하검사(TSI) 무부하검사(급가속) 무부하검사(정지가동) 부하검사(ASM-Idling) 부하검사(KD-147)   
무부하매연판정1                                                                   
N               NaN     5724.0         NaN              NaN      27614.0   
Y               1.0   283244.0         2.0              3.0     762382.0   
NaN             NaN      295.0         NaN              NaN       8242.0   

                                  
검사방법     부하검사(LUG DOWN)      NaN  
무부하매연판정1                          
N                1271.0      NaN  
Y               42256.0      NaN  
NaN               142.0  22637.0

In [104]:
gas_grp = total.groupby(['배출가스인증번호', '무부하매연판정1', '검사방법'], dropna=False)['무부하매연측정치1'].agg(['count', 'mean', 'min', 'max'])
gas_grp

count       mean    min    max
배출가스인증번호      무부하매연판정1 검사방법                                            
              Y        부하검사(ASM-Idling)      1   0.030000   0.03   0.03
                       부하검사(KD-147)          2   7.500000   4.00  11.00
 7MY-DC-24-72 Y        무부하검사(급가속)            1   2.000000   2.00   2.00
 7MY-HD-24-30 N        부하검사(LUG DOWN)        1  43.000000  43.00  43.00
              Y        무부하검사(급가속)            2   0.500000   0.00   1.00
...                                        ...        ...    ...    ...
NaN           Y        부하검사(KD-147)       4389   3.680565   0.00  25.00
                       부하검사(LUG DOWN)      305   3.442623   0.00  26.00
              NaN      무부하검사(급가속)            0        NaN    NaN    NaN
                       부하검사(KD-147)          0        NaN    NaN    NaN
                       NaN                   0        NaN    NaN    NaN

[815 rows x 4 columns]

In [105]:
gas_vin_grp = total.groupby(['배출가스인증번호', '무부하매연판정1', '검사방법'], dropna=False)['차대번호'].agg(['count'])
gas_vin_grp

count
배출가스인증번호      무부하매연판정1 검사방법                   
              Y        부하검사(ASM-Idling)      1
                       부하검사(KD-147)          2
 7MY-DC-24-72 Y        무부하검사(급가속)            1
 7MY-HD-24-30 N        부하검사(LUG DOWN)        1
              Y        무부하검사(급가속)            2
...                                        ...
NaN           Y        부하검사(KD-147)       4389
                       부하검사(LUG DOWN)      305
              NaN      무부하검사(급가속)            5
                       부하검사(KD-147)         25
                       NaN                 378

[815 rows x 1 columns]

In [107]:
gas_grp_y = total[total['무부하매연판정1'] == 'Y'].groupby(['배출가스인증번호', '무부하매연판정1', '검사방법'], dropna=False)['무부하매연측정치1'].agg(['count', 'mean', 'min', 'max'])
gas_grp_y

count      mean   min    max
배출가스인증번호      무부하매연판정1 검사방법                                          
              Y        부하검사(ASM-Idling)      1  0.030000  0.03   0.03
                       부하검사(KD-147)          2  7.500000  4.00  11.00
 7MY-DC-24-72 Y        무부하검사(급가속)            1  2.000000  2.00   2.00
 7MY-HD-24-30 Y        무부하검사(급가속)            2  0.500000  0.00   1.00
                       부하검사(LUG DOWN)        5  9.200000  1.00  16.00
...                                        ...       ...   ...    ...
FMY-VTK-14-4  Y        부하검사(LUG DOWN)       24  0.458333  0.00   4.00
NaN           Y        무부하검사(TSI)            1  0.000000  0.00   0.00
                       무부하검사(급가속)         1939  2.832388  0.00  41.00
                       부하검사(KD-147)       4389  3.680565  0.00  25.00
                       부하검사(LUG DOWN)      305  3.442623  0.00  26.00

[391 rows x 4 columns]

In [108]:
gas_grp_y_vin = total[total['무부하매연판정1'] == 'Y'].groupby(['배출가스인증번호', '무부하매연판정1', '검사방법'], dropna=False)['차대번호'].agg('count')
gas_grp_y_vin

배출가스인증번호       무부하매연판정1  검사방법            
               Y         부하검사(ASM-Idling)       1
                         부하검사(KD-147)           2
 7MY-DC-24-72  Y         무부하검사(급가속)             1
 7MY-HD-24-30  Y         무부하검사(급가속)             2
                         부하검사(LUG DOWN)         5
                                             ... 
FMY-VTK-14-4   Y         부하검사(LUG DOWN)        24
NaN            Y         무부하검사(TSI)             1
                         무부하검사(급가속)          1939
                         부하검사(KD-147)        4389
                         부하검사(LUG DOWN)       305
Name: 차대번호, Length: 391, dtype: int64

In [110]:
gas_grp_n = total[total['무부하매연판정1'] == 'N'].groupby(['배출가스인증번호', '무부하매연판정1', '검사방법'], dropna=False)['무부하매연측정치1'].agg(['count', 'mean', 'min', 'max'])
gas_grp_n

,,,count,mean,min,max
배출가스인증번호,무부하매연판정1,검사방법,,,,
7MY-HD-24-30,N,부하검사(LUG DOWN),1,43.000000,43.0,43.0
4MY-AD-20,N,무부하검사(급가속),1,51.000000,51.0,51.0
5MY-AD-13-06,N,무부하검사(급가속),1,51.000000,51.0,51.0
5MY-AD-14-08,N,부하검사(KD-147),1,65.000000,65.0,65.0
5MY-AD-14-18,N,무부하검사(급가속),1,45.000000,45.0,45.0
...,...,...,...,...,...,...
C135,N,부하검사(KD-147),1,53.000000,53.0,53.0
FMY-VTK-14-1,N,부하검사(LUG DOWN),4,24.750000,18.0,38.0
NaN,N,무부하검사(급가속),6,43.833333,16.0,91.0


In [111]:
gas_grp_n_vin = total[total['무부하매연판정1'] == 'N'].groupby(['배출가스인증번호', '무부하매연판정1', '검사방법'], dropna=False)['차대번호'].agg('count')
gas_grp_n_vin

배출가스인증번호       무부하매연판정1  검사방법          
 7MY-HD-24-30  N         부하검사(LUG DOWN)     1
4MY-AD-20      N         무부하검사(급가속)         1
5MY-AD-13-06   N         무부하검사(급가속)         1
5MY-AD-14-08   N         부하검사(KD-147)       1
5MY-AD-14-18   N         무부하검사(급가속)         1
                                           ..
C135           N         부하검사(KD-147)       1
FMY-VTK-14-1   N         부하검사(LUG DOWN)     4
NaN            N         무부하검사(급가속)         6
                         부하검사(KD-147)      96
                         부하검사(LUG DOWN)     3
Name: 차대번호, Length: 167, dtype: int64

In [112]:
pd.merge(gas_grp_y.reset_index(), gas_grp_n.reset_index(), on='배출가스인증번호', how='outer')

,배출가스인증번호,무부하매연판정1_x,검사방법_x,count_x,mean_x,min_x,max_x,무부하매연판정1_y,검사방법_y,count_y,mean_y,min_y,max_y
0,,Y,부하검사(ASM-Idling),1.0,0.030000,0.03,0.03,NaN,NaN,NaN,NaN,NaN,NaN
1,,Y,부하검사(KD-147),2.0,7.500000,4.00,11.00,NaN,NaN,NaN,NaN,NaN,NaN
2,7MY-DC-24-72,Y,무부하검사(급가속),1.0,2.000000,2.00,2.00,NaN,NaN,NaN,NaN,NaN,NaN
3,7MY-HD-24-30,Y,무부하검사(급가속),2.0,0.500000,0.00,1.00,N,부하검사(LUG DOWN),1.0,43.000000,43.0,43.0
4,7MY-HD-24-30,Y,부하검사(LUG DOWN),5.0,9.200000,1.00,16.00,N,부하검사(LUG DOWN),1.0,43.000000,43.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,NaN,Y,부하검사(KD-147),4389.0,3.680565,0.00,25.00,N,부하검사(LUG DOWN),3.0,17.666667,17.0,19.0
543,NaN,Y,부하검사(LUG DOWN),305.0,3.442623,0.00,26.00,N,무부하검사(급가속),6.0,43.833333,16.0,91.0
544,NaN,Y,부하검사(LUG DOWN),305.0,3.442623,0.00,26.00,N,부하검사(KD-147),96.0,36.395833,10.0,94.0
545,NaN,Y,부하검사(LUG DOWN),305.0,3.442623,0.00,26.00,N,부하검사(LUG DOWN),3.0,17.666667,17.0,19.0


In [113]:
with pd.ExcelWriter('analysis2/4등급_데이터_점검/[BD1][통계][G4]배인번호별_검사방법별_매연측정치_통계.xlsx') as writer:
    # 전체 합격/불합격별 대수 통계
    gas_total_group.to_excel(writer, sheet_name='판정별_통계')
    # 배출가스인증번호, 판정별 매연측정치 통계
    gas_grp.to_excel(writer, sheet_name = '매연측정치_통계')
    gas_vin_grp.to_excel(writer, sheet_name = '매연측정치_차대번호_통계')
    # 매연판정 합격/불합격별 매연측정치 통계
    gas_grp_y.to_excel(writer, sheet_name='합격_매연측정치_통계')
    gas_grp_y_vin.to_excel(writer, sheet_name='합격_매연측정치_차대번호_통계')
    gas_grp_n.to_excel(writer, sheet_name='불합격_매연측정치_통계')
    gas_grp_n_vin.to_excel(writer, sheet_name='불합격_매연측정치_차대번호_통계')
    # 합격/불합격 매연측정치 비교 통계
    pd.merge(gas_grp_y.reset_index(), gas_grp_n.reset_index(), on='배출가스인증번호', how='outer').to_excel(writer, sheet_name='합격_불합격_매연측정치_통계_비교')